This code takes as input the bolometric luminosity data output by Matt Nicholl's superbol code (as a text file named in the format like 'bol_iPTF13bvn_UBVI.txt') and converts it for input into Wynn Jacobson-Galan's pseudo-bolometric light curve fitting code 'Bateman-Eqn-Pseudo-Bolo-LC-Fit.ipynb'.  It also requires the discovery date for each object and the desired filename for the output file, which will be a csv in the format '{filename}_bol.txt' .  It first selects only data with a phase of 50 days or later past the discovery date for analysis.   It then removes the luminosity data with blackbody correction, converts the observed luminosity data and uncertainty to log space, and converts the observation date to a phase by subtracting the discovery date of the SN.

In [18]:
#import dependencies
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import csv
from astropy.time import Time

In [19]:
#read in the csv files; specify tab delimited and skip the irrelevant 10 lines at the bottom of the file
iPTF13bvn = pd.read_csv('bol_iPTF13bvn_UBVI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN2006aj = pd.read_csv('bol_SN2006aj_BVRI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN2007uy = pd.read_csv('bol_SN2007uy_VI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN1993J = pd.read_csv('bol_SN1993J_UBVRI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
iPTF13bvnlt = pd.read_csv('bol_iPTF13bvnlt_UBVI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN1993Jlt = pd.read_csv('bol_SN1993Jlt_UBVRI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN2007uylt = pd.read_csv('bol_SN2007uylt_VI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)
SN2006ajlt = pd.read_csv('bol_SN2006ajlt_BVRI.txt', engine='python', delimiter='\t',header=0, skipfooter=10)

In [ ]:
#select data 50 days or later past discovery
iPTF13bvn = iPTF13bvn[iPTF13bvn.ph >=iPTF13bvn.ph[0]+50]
SN2006aj = SN2006aj[SN2006aj.ph >=SN2006aj.ph[0]+50]
SN2007uy = SN2007uy[SN2007uy.ph >=SN2007uy.ph[0]+50]
SN1993J = SN1993J[SN1993J.ph >=SN1993J.ph[0]+50]

In [12]:
#define a function for selecting and converting the data we want
def convert(data,date_dsc,fname):
    #drop columns for luminosity data & uncertainty w/blackbody correction
    data = data.drop(['L+BB','errbb'],axis=1)
    #convert date to phase = days past discovery
    data.ph = data.ph - date_dsc
    #convert uncertainty to log space using propogation of error
    data.err = np.log10(np.e)/(data['Lobs'])*(data.err)
    #convert data to log space 
    data['Lobs'] = np.log10(data['Lobs'])
    #write to a new csv file
    data.to_csv(str(fname) + '_bol.txt',index=False)

In [6]:
#evaluate the convert function for several objects
convert(iPTF13bvn,56459,'iPTF13bvn')
convert(iPTF13bvnlt,56459,'iPTF13bvnlt')
convert(SN1993J,49074,'1993J')
convert(SN1993Jlt,49074,'1993Jlt')
convert(SN2006aj,53784,'2006aj')
convert(SN2006ajlt,53784,'2006ajlt')
convert(SN2007uy,54465,'2007uy')
convert(SN2007uylt,54465,'2007uylt')

In [13]:
#this does the same thing as the previous function, but it appends the late time data to the early time data first
def convert_both(data,data_lt,date_dsc,fname):
    #append late time data to early time data
    data = data.append(data_lt)
    #drop columns for luminosity data & uncertainty w/blackbody correction
    data = data.drop(['L+BB','errbb'],axis=1)
    #convert date to phase = days past discovery
    data.ph = data.ph - date_dsc
    #convert uncertainty to log space using propogation of error
    data.err = np.log10(np.e)/(data['Lobs'])*(data.err)
    #convert data to log space 
    data['Lobs'] = np.log10(data['Lobs'])
    #write to a new csv file
    data.to_csv(str(fname) + '_bol.txt',index=False)

In [14]:
#evaluate the convert_both function for several objects
convert_both(iPTF13bvn,iPTF13bvnlt,56459,'iPTF13bvn_both')
convert_both(SN1993J,SN1993Jlt,49074,'1993J_both')
convert_both(SN2006aj,SN2006ajlt,53784,'2006aj_both')
convert_both(SN2007uy,SN2007uylt,54465,'2007uy_both')